###### Credit : superdatascience.com

Business case : Classify possibly-churned customer using features of the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import tensorflow as tf

## Data

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

## Preprocessing

##### Missing Data

In [4]:
df = pd.DataFrame(data = X)

In [5]:
df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

##### Categorical Data Encoding - Gender

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

##### Categorical Data Encoding - Country

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X[:2, :])

[[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]
 [0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]]


##### Set Split

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

##### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Artificial Neural Network

##### Model

In [12]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) 
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) 

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

##### Training

In [13]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100


250/250 [==============================] - 1s 850us/step - loss: 0.5199 - accuracy: 0.7919
Epoch 2/100
250/250 [==============================] - 0s 762us/step - loss: 0.4665 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 692us/step - loss: 0.4516 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 648us/step - loss: 0.4434 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 658us/step - loss: 0.4382 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 643us/step - loss: 0.4339 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 656us/step - loss: 0.4302 - accuracy: 0.7961
Epoch 8/100
250/250 [==============================] - 0s 645us/step - loss: 0.4261 - accuracy: 0.8054
Epoch 9/100
250/250 [==============================] - 0s 639us/step - loss: 0.4208 - accuracy: 0.8149
Epoch 10/100
250/250 [==============================] - 0s 662us/step -

250/250 [==============================] - 0s 644us/step - loss: 0.3417 - accuracy: 0.8615
Epoch 76/100
250/250 [==============================] - 0s 646us/step - loss: 0.3410 - accuracy: 0.8608
Epoch 77/100
250/250 [==============================] - 0s 651us/step - loss: 0.3409 - accuracy: 0.8601
Epoch 78/100
250/250 [==============================] - 0s 653us/step - loss: 0.3410 - accuracy: 0.8593
Epoch 79/100
250/250 [==============================] - 0s 682us/step - loss: 0.3408 - accuracy: 0.8625
Epoch 80/100
250/250 [==============================] - 0s 760us/step - loss: 0.3409 - accuracy: 0.8599
Epoch 81/100
250/250 [==============================] - 0s 660us/step - loss: 0.3402 - accuracy: 0.8626
Epoch 82/100
250/250 [==============================] - 0s 755us/step - loss: 0.3404 - accuracy: 0.8608
Epoch 83/100
250/250 [==============================] - 0s 648us/step - loss: 0.3400 - accuracy: 0.8593
Epoch 84/100
250/250 [==============================] - 0s 667us/step - loss:

##### Single Observation Prediction

###### Predict whether the customer with the following informations will leave the bank :
* Geography : France
* Credit Score : 600
* Gender : Male
* Age : 40 yrs old
* Tenure : 3 years
* Balance : usd 60,000
* Number of Products : 2
* Does this customer have a credit card? Yes
* Is this customer an active member? Yes
* Estimated Salary : $ 50,000

In [14]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))     # We get probability (sigmoid)

1/1 [==============================] - 0s 78ms/step
[[0.0445172]]


In [15]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# If you want a boolean value, 0.5 is a assigned threshold
# The customer will probabily stay

1/1 [==============================] - 0s 13ms/step
[[False]]


##### Prediction

In [16]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 575us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


##### Evaluation

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1505   90]
 [ 185  220]]


0.8625